In [1]:
from STvAR_v3 import *
from STvARMetricObjects import *
from STvARFiniteDifference import *
from STvARAdvection import *
from STvARSymbolParsing import *
from STvARIndexing import*

In [2]:
p0r = stvar('p0r', gridvar = True, vartype = 'state_fab', isreal=True)
p0i = stvar('p0i', gridvar = True, vartype = 'state_fab', isreal=True)

p1r = stvar('p1r', gridvar = True, vartype = 'state_fab', isreal=True)
p1i = stvar('p1i', gridvar = True, vartype = 'state_fab', isreal=True)

p_U = stvarrank1('p_U', gridvar = True, vartype = 'state_fab', dim = 2)
p_U.isymb[0] = p0r.isymb + sp.I*p0i.isymb
p_U.isymb[1] = p1r.isymb + sp.I*p1i.isymb

u0r = stvar('u0r', gridvar = True, vartype = 'state_fab', isreal=True)
u0i = stvar('u0i', gridvar = True, vartype = 'state_fab', isreal=True)

u1r = stvar('u0r', gridvar = True, vartype = 'state_fab', isreal=True)
u1i = stvar('u0i', gridvar = True, vartype = 'state_fab', isreal=True)

u_U = stvarrank1('u_U', gridvar = True, vartype = 'state_fab', dim = 2)
u_U.isymb[0] = u0r.isymb + sp.I*u0i.isymb
u_U.isymb[1] = u1r.isymb + sp.I*u1i.isymb

In [3]:
gamma_1 = sp.Matrix([[0,1],[1,0]])
gamma_2 = sp.Matrix([[0,-sp.I],[sp.I,0]])
gamma_3 = sp.Matrix([[1,0],[0,-1]])

In [4]:
print(sp.im(p_U.isymb[0]))

state_fab[i, j, k, state_fabp0i]


In [5]:
print(sp.re(shift(p_U.isymb[0],[1,0,0,0])))

state_fab[i + 1, j, k, state_fabp0r]


In [6]:
print(p_U.isymb[0])

I*state_fab[i, j, k, state_fabp0i] + state_fab[i, j, k, state_fabp0r]


In [7]:
def sigma1Psi(Psi_U):
    sigma1Psi_U = stvarrank1('sigma1Psi_U', dim = 2)
    
    sigma1Psi_U_tmp = gamma_1*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma1Psi_U.isymb[0] = sigma1Psi_U_tmp[0]
    sigma1Psi_U.isymb[1] = sigma1Psi_U_tmp[1]
    
    return sigma1Psi_U

def sigma2Psi(Psi_U):
    sigma2Psi_U = stvarrank1('sigma1Psi_U', dim = 2)
    
    sigma2Psi_U_tmp = gamma_2*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma2Psi_U.isymb[0] = sigma2Psi_U_tmp[0]
    sigma2Psi_U.isymb[1] = sigma2Psi_U_tmp[1]
    
    return sigma1Psi_U

def sigma3Psi(Psi_U):
    sigma3Psi_U = stvarrank1('sigma1Psi_U', dim = 2)
    
    sigma3Psi_U_tmp = gamma_3*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma3Psi_U.isymb[0] = sigma3Psi_U_tmp[0]
    sigma3Psi_U.isymb[1] = sigma3Psi_U_tmp[1]
    
    return sigma3Psi_U

In [8]:
def MuDPsi(psi_U,u_U, r):
    MuDpsi_U = stvarrank1('uDpsi_U', dim = 2)
    
    conjugateu_U = stvarrank1('conju_U', dim = 2)
    conjugateu_U.isymb[0] = sp.conjugate(u_U.isymb[0])
    conjugateu_U.isymb[1] = sp.conjugate(u_U.isymb[1])
    
    psi_U_tmp = sp.Matrix([psi_U.isymb[0], psi_U.isymb[1]])
    
    M1 = sp.eye(2)*r - gamma_1
    M2 = sp.eye(2)*r - gamma_2
    
    Mm1 = sp.eye(2)*r + gamma_1
    Mm2 = sp.eye(2)*r + gamma_2
    
    M1psi_U = M1*psi_U_tmp
    M2psi_U = M2*psi_U_tmp
    
    Mm1psi_U = Mm1*psi_U_tmp
    Mm2psi_U = Mm2*psi_U_tmp
    
    MuDpsi_U.isymb[0] = u_U.isymb[0]*shift(M1psi_U[0],[1,0,0,0])+u_U.isymb[1]*shift(M2psi_U[0],[0,1,0,0])
    MuDpsi_U.isymb[1] = u_U.isymb[0]*shift(M1psi_U[1],[1,0,0,0])+u_U.isymb[1]*shift(M2psi_U[1],[0,1,0,0])
    
    MuDpsi_U.isymb[0] += shift(conjugateu_U.isymb[0],[-1,0,0,0])*shift(Mm1psi_U[0],[-1,0,0,0])+shift(conjugateu_U.isymb[1],[0,-1,0,0])*shift(Mm2psi_U[0],[0,-1,0,0])
    MuDpsi_U.isymb[1] += shift(conjugateu_U.isymb[0],[-1,0,0,0])*shift(Mm1psi_U[1],[-1,0,0,0])+shift(conjugateu_U.isymb[1],[0,-1,0,0])*shift(Mm2psi_U[1],[0,-1,0,0])
    
    return MuDpsi_U

In [9]:
r = sp.symbols('r', real = True)

In [10]:
MuDPsi_U = MuDPsi(p_U,u_U,r)

In [11]:
def OpPsi(Psi_U, u_U, m, r):
    OpPsi_U = stvarrank1('OpPsi_U', dim = 2)
    
    OpPsi_U.isymb[0] = (m+2)*Psi_U.isymb[0]
    OpPsi_U.isymb[1] = (m+2)*Psi_U.isymb[1]
    
    MuDPsi_U = MuDPsi(Psi_U, u_U, r)
    
    OpPsi_U.isymb[0] += MuDPsi_U.isymb[0]
    OpPsi_U.isymb[1] += MuDPsi_U.isymb[1]
    
    OpPsi_U.isymb[0] = sp.simplify(OpPsi_U.isymb[0])
    OpPsi_U.isymb[1] = sp.simplify(OpPsi_U.isymb[1])
    
    
    return OpPsi_U
    
    

In [12]:
m = sp.symbols('m', real = True)

In [13]:
OpPsi_U = OpPsi(p_U, u_U, m, r)

In [14]:
print(sp.re(OpPsi_U.isymb[0]))

-r*state_fab[i + 1, j, k, state_fabp0i]*state_fab[i, j, k, state_fabu0i] + r*state_fab[i + 1, j, k, state_fabp0r]*state_fab[i, j, k, state_fabu0r] + r*state_fab[i - 1, j, k, state_fabp0i]*state_fab[i - 1, j, k, state_fabu0i] + r*state_fab[i - 1, j, k, state_fabp0r]*state_fab[i - 1, j, k, state_fabu0r] - r*state_fab[i, j + 1, k, state_fabp0i]*state_fab[i, j, k, state_fabu0i] + r*state_fab[i, j + 1, k, state_fabp0r]*state_fab[i, j, k, state_fabu0r] + r*state_fab[i, j - 1, k, state_fabp0i]*state_fab[i, j - 1, k, state_fabu0i] + r*state_fab[i, j - 1, k, state_fabp0r]*state_fab[i, j - 1, k, state_fabu0r] + (m + 2)*state_fab[i, j, k, state_fabp0r] + state_fab[i + 1, j, k, state_fabp1i]*state_fab[i, j, k, state_fabu0i] - state_fab[i + 1, j, k, state_fabp1r]*state_fab[i, j, k, state_fabu0r] + state_fab[i - 1, j, k, state_fabp1i]*state_fab[i - 1, j, k, state_fabu0i] + state_fab[i - 1, j, k, state_fabp1r]*state_fab[i - 1, j, k, state_fabu0r] - state_fab[i, j + 1, k, state_fabp1i]*state_fab[i, j,

In [15]:
MuDPsi_U = MuDPsi(p_U,u_U,r)

In [16]:
print(sp.re(MuDPsi_U.isymb[0]))

-r*state_fab[i + 1, j, k, state_fabp0i]*state_fab[i, j, k, state_fabu0i] + r*state_fab[i + 1, j, k, state_fabp0r]*state_fab[i, j, k, state_fabu0r] + r*state_fab[i - 1, j, k, state_fabp0i]*state_fab[i - 1, j, k, state_fabu0i] + r*state_fab[i - 1, j, k, state_fabp0r]*state_fab[i - 1, j, k, state_fabu0r] - r*state_fab[i, j + 1, k, state_fabp0i]*state_fab[i, j, k, state_fabu0i] + r*state_fab[i, j + 1, k, state_fabp0r]*state_fab[i, j, k, state_fabu0r] + r*state_fab[i, j - 1, k, state_fabp0i]*state_fab[i, j - 1, k, state_fabu0i] + r*state_fab[i, j - 1, k, state_fabp0r]*state_fab[i, j - 1, k, state_fabu0r] + state_fab[i + 1, j, k, state_fabp1i]*state_fab[i, j, k, state_fabu0i] - state_fab[i + 1, j, k, state_fabp1r]*state_fab[i, j, k, state_fabu0r] + state_fab[i - 1, j, k, state_fabp1i]*state_fab[i - 1, j, k, state_fabu0i] + state_fab[i - 1, j, k, state_fabp1r]*state_fab[i - 1, j, k, state_fabu0r] - state_fab[i, j + 1, k, state_fabp1i]*state_fab[i, j, k, state_fabu0r] - state_fab[i, j + 1, k, 

In [17]:
OpOpPsi_U = OpPsi(OpPsi(p_U,u_U,m,r), u_U,m,r)

In [26]:
OpOpPsiReal_U = stvarrank1('OpOpPsiReal_U', isreal = True, dim = 2)
OpOpPsiReal_U.isymb[0] = sp.re(OpOpPsi_U.isymb[0])
OpOpPsiReal_U.isymb[1] = sp.re(OpOpPsi_U.isymb[1])

OpOpPsiImaginary_U = stvarrank1('OpOpPsiImaginary_U', isreal = True, dim = 2)
OpOpPsiImaginary_U.isymb[0] = sp.im(OpOpPsi_U.isymb[0])
OpOpPsiImaginary_U.isymb[1] = sp.im(OpOpPsi_U.isymb[1])

In [27]:
print(OpOpPsiReal_U.symb2isymb().replace("state_fabIdx","Idx"))

        amrex::Real OpOpPsiReal_U_0 = -r*(r*state_fab(i + 1, j + 1, k, Idx::p0i)*state_fab(i + 1, j, k, Idx::u0i) - r*state_fab(i + 1, j + 1, k, Idx::p0r)*state_fab(i + 1, j, k, Idx::u0r) - r*state_fab(i + 1, j - 1, k, Idx::p0i)*state_fab(i + 1, j - 1, k, Idx::u0i) - r*state_fab(i + 1, j - 1, k, Idx::p0r)*state_fab(i + 1, j - 1, k, Idx::u0r) + r*state_fab(i + 1, j, k, Idx::u0i)*state_fab(i + 2, j, k, Idx::p0i) - r*state_fab(i + 1, j, k, Idx::u0r)*state_fab(i + 2, j, k, Idx::p0r) - r*state_fab(i, j, k, Idx::p0i)*state_fab(i, j, k, Idx::u0i) - r*state_fab(i, j, k, Idx::p0r)*state_fab(i, j, k, Idx::u0r) + (-m - 2)*state_fab(i + 1, j, k, Idx::p0r) + state_fab(i + 1, j + 1, k, Idx::p1i)*state_fab(i + 1, j, k, Idx::u0r) + state_fab(i + 1, j + 1, k, Idx::p1r)*state_fab(i + 1, j, k, Idx::u0i) - state_fab(i + 1, j - 1, k, Idx::p1i)*state_fab(i + 1, j - 1, k, Idx::u0r) + state_fab(i + 1, j - 1, k, Idx::p1r)*state_fab(i + 1, j - 1, k, Idx::u0i) - state_fab(i + 1, j, k, Idx::u0i)*state_fab(i + 2, 